In [1]:
cd ..

/Users/joeranbosma/stack/Projects/M5Forecast


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, LeakyReLU
from tensorflow.keras.layers import Flatten, Input, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, Reshape, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.environ['DATA_DIR'] = 'data/'
os.environ['SUB_DIR'] = 'submissions_uncertainty/'
for dirname, _, filenames in os.walk(os.environ['DATA_DIR']):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/Users/joeranbosma/opt/anaconda3/envs/mlip/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence
/Users/joeranbosma/opt/anaconda3/envs/mlip/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


data/.DS_Store
data/empty_pred.csv
data/pred_skeleton.csv
data/calendar.csv
data/sales_train_money.csv
data/sell_prices.csv
data/sales_train_validation.csv
data/sample_submission.csv


# Pinball Loss function for Keras

In [3]:
train_df = pd.read_csv("custom_layer/features.csv", index_col=0)
target_df = pd.read_csv("custom_layer/targets.csv", index_col=0)

In [4]:
train_df.head()

,wday,month,snap_CA,w_1,w_2,w_3,w_4,w_5,w_6,w_7
date,,,,,,,,,,
2011-01-29,1,1,0,1,0,0,0,0,0,0
2011-01-30,2,1,0,0,1,0,0,0,0,0
2011-01-31,3,1,0,0,0,1,0,0,0,0
2011-02-01,4,2,1,0,0,0,1,0,0,0
2011-02-02,5,2,1,0,0,0,0,1,0,0


In [5]:
target_df.head()

,0.005,0.025,0.165,0.25,0.5,0.75,0.835,0.975,0.995
date,,,,,,,,,
2011-01-29,0.410956,0.494101,0.630373,0.671720,0.756835,0.843188,0.884852,1.006654,1.097859
2011-01-30,0.919469,0.992795,1.270255,1.332988,1.504236,1.675052,1.746716,2.014860,2.188142
2011-01-31,0.790407,0.988427,1.249410,1.322885,1.500305,1.659404,1.720131,1.953909,2.180639
2011-02-01,0.414008,0.487010,0.623712,0.659987,0.747542,0.837403,0.876519,1.000635,1.067878
2011-02-02,0.443139,0.511417,0.619042,0.662347,0.756661,0.838183,0.875509,0.997830,1.074211


In [6]:
def get_model(inp_shape, quantiles):
    # clear previous sessions
    K.clear_session()

    inp = Input(inp_shape, name="input")
    x = inp
    x = Dense(16)(x)
    x = Dense(32)(x)
    x = Dense(64)(x)
    x = Dense(2)(x)  # represents mu, sigma
    x = Dense(len(quantiles))(x)  # returns 7 points, one for each quantile
    out = x

    model = Model(inputs=inp, outputs=out)

    return model

In [7]:
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]
model = get_model(inp_shape=(train_df.columns.size,), quantiles=quantiles)
model.compile(optimizer="adam", loss="MAE")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 16)                176       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 27        
Total params: 2,989
Trainable params: 2,989
Non-trainable params: 0
___________________________________________________________

In [8]:
X = train_df.values
y = target_df.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_val, y_val))

Epoch 1/100
9/9 [==============================] - 0s 15ms/step - loss: 1.0342 - val_loss: 0.9296
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9713 - val_loss: 0.8921
Epoch 3/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9482 - val_loss: 0.8745
Epoch 4/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9289 - val_loss: 0.8560
Epoch 5/100
9/9 [==============================] - 0s 4ms/step - loss: 0.9103 - val_loss: 0.8367
Epoch 6/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8911 - val_loss: 0.8240
Epoch 7/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8788 - val_loss: 0.8141
Epoch 8/100
9/9 [==============================] - 0s 4ms/step - loss: 0.8616 - val_loss: 0.7945
Epoch 9/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8433 - val_loss: 0.7767
Epoch 10/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8262 - val_loss: 0.7586
Epoch 11/100
9/9 [==========

9/9 [==============================] - 0s 4ms/step - loss: 0.0215 - val_loss: 0.0301
Epoch 85/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0252 - val_loss: 0.0310
Epoch 86/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0323 - val_loss: 0.0276
Epoch 87/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0271 - val_loss: 0.0516
Epoch 88/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0434 - val_loss: 0.0450
Epoch 89/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0426 - val_loss: 0.0448
Epoch 90/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0337 - val_loss: 0.0363
Epoch 91/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0235 - val_loss: 0.0221
Epoch 92/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0217 - val_loss: 0.0241
Epoch 93/100
9/9 [==============================] - 0s 4ms/step - loss: 0.0230 - val_loss: 0.0258
Epoch 94/100
9/9 [===============

In [10]:
y_pred = model.predict(X_val)

In [11]:
y_pred

array([[0.4243765 , 0.48935676, 0.6214396 , 0.68627584, 0.77260983,
        0.85994625, 0.87027377, 1.0282639 , 1.086638  ],
       [0.87302256, 0.9871994 , 1.2360862 , 1.3489847 , 1.5108074 ,
        1.6816196 , 1.7309608 , 2.006692  , 2.1233525 ],
       [0.87694734, 0.99236935, 1.2422463 , 1.353865  , 1.5165588 ,
        1.6880648 , 1.7391102 , 2.0143836 , 2.1330104 ],
       [0.42473856, 0.49046803, 0.6226175 , 0.6860122 , 0.77259064,
        0.8599625 , 0.87150836, 1.028298  , 1.0879873 ],
       [0.87622315, 0.9901472 , 1.2398908 , 1.354392  , 1.5165972 ,
        1.6880322 , 1.7366414 , 2.0143151 , 2.1303122 ],
       [0.8726605 , 0.9860886 , 1.2349088 , 1.3492479 , 1.5108263 ,
        1.6816032 , 1.7297266 , 2.0066574 , 2.1220038 ],
       [0.41975486, 0.48192042, 0.6128901 , 0.68204635, 0.7670063 ,
        0.8535864 , 0.85965115, 1.0206428 , 1.0742911 ],
       [0.4204033 , 0.49042946, 0.62126386, 0.674238  , 0.76132035,
        0.8476715 , 0.86682326, 1.0137614 , 1.0814172 ],


## Rewrite to multi-output for each quantile

In [12]:
def get_model(inp_shape, quantiles):
    # clear previous sessions
    K.clear_session()

    inp = Input(inp_shape, name="input")
    x = inp
    x = Dense(16)(x)
    x = Dense(32)(x)
    x = Dense(64)(x)
    x = Dense(2)(x)  # represents mu, sigma
    
    out_q0 = Dense(1, name="q0")(x)  # DistributionLayer(quantile=quantiles[0])(x)
    out_q1 = Dense(1, name="q1")(x)  # DistributionLayer(quantile=quantiles[1])(x)
    out_q2 = Dense(1, name="q2")(x)  # ...
    out_q3 = Dense(1, name="q3")(x)
    out_q4 = Dense(1, name="q4")(x)
    out_q5 = Dense(1, name="q5")(x)
    out_q6 = Dense(1, name="q6")(x)
    out_q7 = Dense(1, name="q7")(x)
    out_q8 = Dense(1, name="q8")(x)

    model = Model(inputs=inp, outputs=[out_q0, out_q1, out_q2, out_q3, out_q4, out_q5, out_q6, out_q7, out_q8])

    return model

In [13]:
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]
model = get_model(inp_shape=(train_df.columns.size,), quantiles=quantiles)
model.compile(optimizer="adam", loss="MAE")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 16)           176         input[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           544         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           2112        dense_1[0][0]                    
______________________________________________________________________________________________

In [14]:
y_train_mo = {'q'+str(i): y_train[:, i] for i in range(len(quantiles))}
y_val_mo = {'q'+str(i): y_val[:, i] for i in range(len(quantiles))}
y_train_mo

{'q0': array([0.81128298, 0.36194897, 0.431012  , 0.41013684, 0.85977491,
        0.42128552, 0.42166976, 0.40354603, 0.47146557, 0.42250803,
        0.40243837, 0.42686927, 0.44992118, 0.83710284, 0.42648037,
        0.40455794, 0.45533506, 0.4416203 , 0.44018187, 0.41073501,
        0.39531865, 0.82886827, 0.37881828, 0.40348279, 0.86927252,
        0.42722338, 0.80450429, 0.42121954, 0.41807985, 0.87185556,
        0.87785825, 0.85858611, 0.41152437, 0.41251877, 0.87180258,
        0.43765844, 0.41462572, 0.4143409 , 0.85627792, 0.92359907,
        0.84874455, 0.44157534, 0.40350371, 0.40480417, 0.40318321,
        0.40144885, 0.44090451, 0.85685899, 0.4335312 , 0.42025695,
        0.42924569, 0.88817238, 0.41945536, 0.43180025, 0.39893377,
        0.89810435, 0.40669158, 0.40587732, 0.42109244, 0.42658644,
        0.45097504, 0.40643598, 0.87694607, 0.41230595, 0.88223235,
        0.38546639, 0.44442082, 0.41613937, 0.42775111, 0.90421291,
        0.83740529, 0.87137531, 0.38811858

In [15]:
history = model.fit(X_train, y_train_mo, epochs=100,
                    validation_data=(X_val, y_val_mo))

Epoch 1/100
9/9 [==============================] - 0s 37ms/step - loss: 10.6101 - q0_loss: 0.9080 - q1_loss: 1.8156 - q2_loss: 1.3543 - q3_loss: 1.4921 - q4_loss: 0.7092 - q5_loss: 0.8334 - q6_loss: 0.9505 - q7_loss: 1.2818 - q8_loss: 1.2652 - val_loss: 8.1568 - val_q0_loss: 0.3739 - val_q1_loss: 1.6262 - val_q2_loss: 1.1914 - val_q3_loss: 1.5725 - val_q4_loss: 0.4265 - val_q5_loss: 0.5017 - val_q6_loss: 0.6820 - val_q7_loss: 0.5174 - val_q8_loss: 1.2653
Epoch 2/100
9/9 [==============================] - 0s 6ms/step - loss: 8.2825 - q0_loss: 0.6995 - q1_loss: 0.7331 - q2_loss: 0.5983 - q3_loss: 1.7760 - q4_loss: 0.5457 - q5_loss: 0.7699 - q6_loss: 0.9577 - q7_loss: 1.1348 - q8_loss: 1.0676 - val_loss: 7.4295 - val_q0_loss: 0.6408 - val_q1_loss: 0.4936 - val_q2_loss: 0.4666 - val_q3_loss: 1.4229 - val_q4_loss: 0.4007 - val_q5_loss: 0.8241 - val_q6_loss: 0.9638 - val_q7_loss: 1.1690 - val_q8_loss: 1.0479
Epoch 3/100
9/9 [==============================] - 0s 6ms/step - loss: 8.0532 - q0_l

Epoch 19/100
9/9 [==============================] - 0s 6ms/step - loss: 4.2918 - q0_loss: 0.1889 - q1_loss: 0.3834 - q2_loss: 0.0533 - q3_loss: 1.8994 - q4_loss: 0.2054 - q5_loss: 0.1624 - q6_loss: 0.4079 - q7_loss: 0.5021 - q8_loss: 0.4890 - val_loss: 3.7843 - val_q0_loss: 0.1515 - val_q1_loss: 0.3175 - val_q2_loss: 0.0304 - val_q3_loss: 1.7657 - val_q4_loss: 0.1748 - val_q5_loss: 0.1164 - val_q6_loss: 0.3571 - val_q7_loss: 0.4464 - val_q8_loss: 0.4245
Epoch 20/100
9/9 [==============================] - 0s 6ms/step - loss: 4.0415 - q0_loss: 0.1595 - q1_loss: 0.3618 - q2_loss: 0.0221 - q3_loss: 1.8939 - q4_loss: 0.1942 - q5_loss: 0.1316 - q6_loss: 0.3692 - q7_loss: 0.4586 - q8_loss: 0.4506 - val_loss: 3.6207 - val_q0_loss: 0.1383 - val_q1_loss: 0.2447 - val_q2_loss: 0.0842 - val_q3_loss: 1.7960 - val_q4_loss: 0.2110 - val_q5_loss: 0.0799 - val_q6_loss: 0.3084 - val_q7_loss: 0.4090 - val_q8_loss: 0.3491
Epoch 21/100
9/9 [==============================] - 0s 6ms/step - loss: 3.8474 - q0_

Epoch 37/100
9/9 [==============================] - 0s 6ms/step - loss: 1.9486 - q0_loss: 0.0528 - q1_loss: 0.0591 - q2_loss: 0.0272 - q3_loss: 1.5257 - q4_loss: 0.1535 - q5_loss: 0.0249 - q6_loss: 0.0312 - q7_loss: 0.0436 - q8_loss: 0.0306 - val_loss: 1.7847 - val_q0_loss: 0.0349 - val_q1_loss: 0.0604 - val_q2_loss: 0.0304 - val_q3_loss: 1.3423 - val_q4_loss: 0.1661 - val_q5_loss: 0.0431 - val_q6_loss: 0.0279 - val_q7_loss: 0.0374 - val_q8_loss: 0.0421
Epoch 38/100
9/9 [==============================] - 0s 5ms/step - loss: 1.8825 - q0_loss: 0.0454 - q1_loss: 0.0640 - q2_loss: 0.0194 - q3_loss: 1.4873 - q4_loss: 0.1602 - q5_loss: 0.0235 - q6_loss: 0.0228 - q7_loss: 0.0305 - q8_loss: 0.0296 - val_loss: 1.6989 - val_q0_loss: 0.0448 - val_q1_loss: 0.0528 - val_q2_loss: 0.0260 - val_q3_loss: 1.3526 - val_q4_loss: 0.1383 - val_q5_loss: 0.0163 - val_q6_loss: 0.0210 - val_q7_loss: 0.0290 - val_q8_loss: 0.0181
Epoch 39/100
9/9 [==============================] - 0s 6ms/step - loss: 1.8583 - q0_

Epoch 55/100
9/9 [==============================] - 0s 6ms/step - loss: 1.3459 - q0_loss: 0.0453 - q1_loss: 0.0406 - q2_loss: 0.0159 - q3_loss: 0.9682 - q4_loss: 0.1327 - q5_loss: 0.0297 - q6_loss: 0.0327 - q7_loss: 0.0485 - q8_loss: 0.0322 - val_loss: 1.1988 - val_q0_loss: 0.0390 - val_q1_loss: 0.0481 - val_q2_loss: 0.0093 - val_q3_loss: 0.8459 - val_q4_loss: 0.1243 - val_q5_loss: 0.0275 - val_q6_loss: 0.0307 - val_q7_loss: 0.0434 - val_q8_loss: 0.0307
Epoch 56/100
9/9 [==============================] - 0s 6ms/step - loss: 1.3021 - q0_loss: 0.0419 - q1_loss: 0.0473 - q2_loss: 0.0214 - q3_loss: 0.9449 - q4_loss: 0.1322 - q5_loss: 0.0244 - q6_loss: 0.0247 - q7_loss: 0.0397 - q8_loss: 0.0256 - val_loss: 1.1304 - val_q0_loss: 0.0403 - val_q1_loss: 0.0306 - val_q2_loss: 0.0226 - val_q3_loss: 0.8397 - val_q4_loss: 0.1143 - val_q5_loss: 0.0114 - val_q6_loss: 0.0206 - val_q7_loss: 0.0307 - val_q8_loss: 0.0201
Epoch 57/100
9/9 [==============================] - 0s 6ms/step - loss: 1.2936 - q0_

Epoch 73/100
9/9 [==============================] - 0s 6ms/step - loss: 1.0012 - q0_loss: 0.0354 - q1_loss: 0.0742 - q2_loss: 0.0689 - q3_loss: 0.4919 - q4_loss: 0.1212 - q5_loss: 0.0412 - q6_loss: 0.0444 - q7_loss: 0.0463 - q8_loss: 0.0777 - val_loss: 0.8382 - val_q0_loss: 0.0279 - val_q1_loss: 0.0999 - val_q2_loss: 0.0730 - val_q3_loss: 0.4023 - val_q4_loss: 0.0636 - val_q5_loss: 0.0279 - val_q6_loss: 0.0378 - val_q7_loss: 0.0347 - val_q8_loss: 0.0711
Epoch 74/100
9/9 [==============================] - 0s 5ms/step - loss: 0.8654 - q0_loss: 0.0322 - q1_loss: 0.0892 - q2_loss: 0.0588 - q3_loss: 0.4642 - q4_loss: 0.0959 - q5_loss: 0.0208 - q6_loss: 0.0235 - q7_loss: 0.0327 - q8_loss: 0.0481 - val_loss: 0.8350 - val_q0_loss: 0.0387 - val_q1_loss: 0.0627 - val_q2_loss: 0.0624 - val_q3_loss: 0.3859 - val_q4_loss: 0.1030 - val_q5_loss: 0.0363 - val_q6_loss: 0.0404 - val_q7_loss: 0.0428 - val_q8_loss: 0.0627
Epoch 75/100
9/9 [==============================] - 0s 6ms/step - loss: 0.8093 - q0_

Epoch 91/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6340 - q0_loss: 0.0464 - q1_loss: 0.0755 - q2_loss: 0.0499 - q3_loss: 0.2415 - q4_loss: 0.0687 - q5_loss: 0.0303 - q6_loss: 0.0302 - q7_loss: 0.0518 - q8_loss: 0.0398 - val_loss: 0.6970 - val_q0_loss: 0.0420 - val_q1_loss: 0.1289 - val_q2_loss: 0.1077 - val_q3_loss: 0.1775 - val_q4_loss: 0.0897 - val_q5_loss: 0.0184 - val_q6_loss: 0.0261 - val_q7_loss: 0.0247 - val_q8_loss: 0.0818
Epoch 92/100
9/9 [==============================] - 0s 5ms/step - loss: 0.5599 - q0_loss: 0.0319 - q1_loss: 0.0673 - q2_loss: 0.0489 - q3_loss: 0.2378 - q4_loss: 0.0756 - q5_loss: 0.0156 - q6_loss: 0.0165 - q7_loss: 0.0275 - q8_loss: 0.0388 - val_loss: 0.4118 - val_q0_loss: 0.0336 - val_q1_loss: 0.0288 - val_q2_loss: 0.0215 - val_q3_loss: 0.1761 - val_q4_loss: 0.0629 - val_q5_loss: 0.0166 - val_q6_loss: 0.0184 - val_q7_loss: 0.0324 - val_q8_loss: 0.0214
Epoch 93/100
9/9 [==============================] - 0s 5ms/step - loss: 0.5184 - q0_

In [16]:
y_pred = model.predict(X_val)

# revert multi output format to (n_samples, quantiles)
y_pred = np.array(list(zip(*y_pred))).squeeze()
y_pred

array([[0.42806643, 0.5204087 , 0.6345039 , 0.6815528 , 0.71727407,
        0.84872127, 0.89159036, 1.0215573 , 1.1024837 ],
       [0.823116  , 1.0038371 , 1.2524054 , 0.62017083, 1.5690337 ,
        1.6344151 , 1.7058115 , 1.9322469 , 2.1132894 ],
       [0.8481551 , 0.97053367, 1.2352908 , 0.60762966, 1.5839158 ,
        1.655868  , 1.7252618 , 1.9684604 , 2.1189203 ],
       [0.4333601 , 0.5194798 , 0.63626486, 0.67972827, 0.72415817,
        0.8559662 , 0.8987761 , 1.0312692 , 1.1092596 ],
       [0.837568  , 0.9723913 , 1.2317691 , 0.61127865, 1.5701482 ,
        1.6413788 , 1.7108907 , 1.9490372 , 2.1053689 ],
       [0.8178223 , 1.0047656 , 1.250644  , 0.6219952 , 1.5621495 ,
        1.6271702 , 1.6986256 , 1.9225348 , 2.1065128 ],
       [0.41478992, 0.5237179 , 0.6309493 , 0.6862613 , 0.700608  ,
        0.8309853 , 0.87406105, 0.9975292 , 1.0863845 ],
       [0.42696005, 0.4910725 , 0.6081457 , 0.67793924, 0.69777656,
        0.8341166 , 0.8752381 , 1.0095948 , 1.0740811 ],


## Employ pinball loss

In [17]:
# from Daniel Sch., at:
# https://stackoverflow.com/questions/43151694/define-pinball-loss-function-in-keras-with-tensorflow-backend
def create_pinball_loss(tau=0.5):
    def pinball_loss(y_true, y_pred):
        err = y_true - y_pred
        return K.mean(K.maximum(tau * err, (tau - 1) * err), axis=-1)
    return pinball_loss

In [18]:
losses = {'q'+str(i): create_pinball_loss(tau=q) for (i, q) in enumerate(quantiles)}
losses

{'q0': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q1': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q2': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q3': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q4': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q5': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q6': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q7': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>,
 'q8': <function __main__.create_pinball_loss.<locals>.pinball_loss(y_true, y_pred)>}

In [19]:
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]
model = get_model(inp_shape=(train_df.columns.size,), quantiles=quantiles)
model.compile(optimizer="adam", loss=losses)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 16)           176         input[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           544         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           2112        dense_1[0][0]                    
______________________________________________________________________________________________

In [20]:
history = model.fit(X_train, y_train, epochs=300,
                    validation_data=(X_val, y_val))

Epoch 1/300
9/9 [==============================] - 0s 32ms/step - loss: 5.8409 - q0_loss: 0.1334 - q1_loss: 0.0904 - q2_loss: 0.2663 - q3_loss: 0.4159 - q4_loss: 0.7627 - q5_loss: 0.8713 - q6_loss: 1.1265 - q7_loss: 1.0403 - q8_loss: 1.1341 - val_loss: 4.2957 - val_q0_loss: 0.0077 - val_q1_loss: 0.3292 - val_q2_loss: 0.2302 - val_q3_loss: 0.3954 - val_q4_loss: 0.3538 - val_q5_loss: 1.1228 - val_q6_loss: 0.4054 - val_q7_loss: 1.0815 - val_q8_loss: 0.3697
Epoch 2/300
9/9 [==============================] - 0s 5ms/step - loss: 3.9737 - q0_loss: 0.0072 - q1_loss: 0.3466 - q2_loss: 0.1622 - q3_loss: 0.2279 - q4_loss: 0.3833 - q5_loss: 0.7009 - q6_loss: 0.3359 - q7_loss: 0.5670 - q8_loss: 1.2427 - val_loss: 3.6862 - val_q0_loss: 0.0071 - val_q1_loss: 0.3139 - val_q2_loss: 0.1153 - val_q3_loss: 0.1562 - val_q4_loss: 0.4971 - val_q5_loss: 0.3819 - val_q6_loss: 0.2474 - val_q7_loss: 0.2676 - val_q8_loss: 1.6997
Epoch 3/300
9/9 [==============================] - 0s 5ms/step - loss: 3.4985 - q0_lo

Epoch 19/300
9/9 [==============================] - 0s 5ms/step - loss: 1.5664 - q0_loss: 0.0061 - q1_loss: 0.0530 - q2_loss: 0.0943 - q3_loss: 0.0943 - q4_loss: 0.1226 - q5_loss: 0.3742 - q6_loss: 0.1749 - q7_loss: 0.1466 - q8_loss: 0.5005 - val_loss: 1.3788 - val_q0_loss: 0.0058 - val_q1_loss: 0.0533 - val_q2_loss: 0.0835 - val_q3_loss: 0.0916 - val_q4_loss: 0.1217 - val_q5_loss: 0.3250 - val_q6_loss: 0.1765 - val_q7_loss: 0.1084 - val_q8_loss: 0.4129
Epoch 20/300
9/9 [==============================] - 0s 4ms/step - loss: 1.4592 - q0_loss: 0.0061 - q1_loss: 0.0480 - q2_loss: 0.0897 - q3_loss: 0.0936 - q4_loss: 0.1237 - q5_loss: 0.3471 - q6_loss: 0.1766 - q7_loss: 0.1203 - q8_loss: 0.4540 - val_loss: 1.2773 - val_q0_loss: 0.0058 - val_q1_loss: 0.0458 - val_q2_loss: 0.0791 - val_q3_loss: 0.0891 - val_q4_loss: 0.1200 - val_q5_loss: 0.2979 - val_q6_loss: 0.1746 - val_q7_loss: 0.0867 - val_q8_loss: 0.3784
Epoch 21/300
9/9 [==============================] - 0s 5ms/step - loss: 1.3527 - q0_

Epoch 37/300
9/9 [==============================] - 0s 4ms/step - loss: 0.5751 - q0_loss: 0.0059 - q1_loss: 0.0173 - q2_loss: 0.0660 - q3_loss: 0.0888 - q4_loss: 0.1145 - q5_loss: 0.1010 - q6_loss: 0.1384 - q7_loss: 0.0121 - q8_loss: 0.0311 - val_loss: 0.5231 - val_q0_loss: 0.0055 - val_q1_loss: 0.0147 - val_q2_loss: 0.0625 - val_q3_loss: 0.0841 - val_q4_loss: 0.1072 - val_q5_loss: 0.0909 - val_q6_loss: 0.1218 - val_q7_loss: 0.0109 - val_q8_loss: 0.0256
Epoch 38/300
9/9 [==============================] - 0s 4ms/step - loss: 0.5600 - q0_loss: 0.0058 - q1_loss: 0.0162 - q2_loss: 0.0659 - q3_loss: 0.0888 - q4_loss: 0.1140 - q5_loss: 0.0991 - q6_loss: 0.1299 - q7_loss: 0.0121 - q8_loss: 0.0282 - val_loss: 0.5104 - val_q0_loss: 0.0055 - val_q1_loss: 0.0142 - val_q2_loss: 0.0621 - val_q3_loss: 0.0838 - val_q4_loss: 0.1071 - val_q5_loss: 0.0914 - val_q6_loss: 0.1136 - val_q7_loss: 0.0117 - val_q8_loss: 0.0212
Epoch 39/300
9/9 [==============================] - 0s 4ms/step - loss: 0.5479 - q0_

Epoch 55/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4691 - q0_loss: 0.0049 - q1_loss: 0.0133 - q2_loss: 0.0657 - q3_loss: 0.0879 - q4_loss: 0.1153 - q5_loss: 0.0902 - q6_loss: 0.0737 - q7_loss: 0.0117 - q8_loss: 0.0064 - val_loss: 0.4389 - val_q0_loss: 0.0046 - val_q1_loss: 0.0128 - val_q2_loss: 0.0621 - val_q3_loss: 0.0828 - val_q4_loss: 0.1084 - val_q5_loss: 0.0847 - val_q6_loss: 0.0675 - val_q7_loss: 0.0109 - val_q8_loss: 0.0050
Epoch 56/300
9/9 [==============================] - 0s 6ms/step - loss: 0.4681 - q0_loss: 0.0049 - q1_loss: 0.0133 - q2_loss: 0.0657 - q3_loss: 0.0879 - q4_loss: 0.1151 - q5_loss: 0.0903 - q6_loss: 0.0734 - q7_loss: 0.0116 - q8_loss: 0.0058 - val_loss: 0.4389 - val_q0_loss: 0.0046 - val_q1_loss: 0.0128 - val_q2_loss: 0.0619 - val_q3_loss: 0.0831 - val_q4_loss: 0.1091 - val_q5_loss: 0.0844 - val_q6_loss: 0.0672 - val_q7_loss: 0.0109 - val_q8_loss: 0.0048
Epoch 57/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4673 - q0_

Epoch 73/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4599 - q0_loss: 0.0036 - q1_loss: 0.0132 - q2_loss: 0.0654 - q3_loss: 0.0879 - q4_loss: 0.1149 - q5_loss: 0.0895 - q6_loss: 0.0708 - q7_loss: 0.0116 - q8_loss: 0.0031 - val_loss: 0.4332 - val_q0_loss: 0.0034 - val_q1_loss: 0.0127 - val_q2_loss: 0.0616 - val_q3_loss: 0.0831 - val_q4_loss: 0.1085 - val_q5_loss: 0.0841 - val_q6_loss: 0.0661 - val_q7_loss: 0.0108 - val_q8_loss: 0.0030
Epoch 74/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4598 - q0_loss: 0.0035 - q1_loss: 0.0132 - q2_loss: 0.0652 - q3_loss: 0.0880 - q4_loss: 0.1150 - q5_loss: 0.0893 - q6_loss: 0.0708 - q7_loss: 0.0115 - q8_loss: 0.0031 - val_loss: 0.4330 - val_q0_loss: 0.0033 - val_q1_loss: 0.0127 - val_q2_loss: 0.0618 - val_q3_loss: 0.0829 - val_q4_loss: 0.1083 - val_q5_loss: 0.0842 - val_q6_loss: 0.0660 - val_q7_loss: 0.0109 - val_q8_loss: 0.0029
Epoch 75/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4598 - q0_

...

Epoch 217/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4518 - q0_loss: 0.0025 - q1_loss: 0.0121 - q2_loss: 0.0654 - q3_loss: 0.0880 - q4_loss: 0.1143 - q5_loss: 0.0890 - q6_loss: 0.0656 - q7_loss: 0.0118 - q8_loss: 0.0033 - val_loss: 0.4246 - val_q0_loss: 0.0023 - val_q1_loss: 0.0116 - val_q2_loss: 0.0615 - val_q3_loss: 0.0831 - val_q4_loss: 0.1075 - val_q5_loss: 0.0834 - val_q6_loss: 0.0615 - val_q7_loss: 0.0107 - val_q8_loss: 0.0031
Epoch 218/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4518 - q0_loss: 0.0024 - q1_loss: 0.0121 - q2_loss: 0.0653 - q3_loss: 0.0881 - q4_loss: 0.1143 - q5_loss: 0.0889 - q6_loss: 0.0657 - q7_loss: 0.0118 - q8_loss: 0.0033 - val_loss: 0.4250 - val_q0_loss: 0.0023 - val_q1_loss: 0.0115 - val_q2_loss: 0.0615 - val_q3_loss: 0.0831 - val_q4_loss: 0.1077 - val_q5_loss: 0.0834 - val_q6_loss: 0.0614 - val_q7_loss: 0.0107 - val_q8_loss: 0.0033
Epoch 219/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4528 - 

Epoch 235/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4538 - q0_loss: 0.0026 - q1_loss: 0.0119 - q2_loss: 0.0658 - q3_loss: 0.0884 - q4_loss: 0.1150 - q5_loss: 0.0891 - q6_loss: 0.0656 - q7_loss: 0.0121 - q8_loss: 0.0033 - val_loss: 0.4249 - val_q0_loss: 0.0023 - val_q1_loss: 0.0113 - val_q2_loss: 0.0615 - val_q3_loss: 0.0832 - val_q4_loss: 0.1075 - val_q5_loss: 0.0835 - val_q6_loss: 0.0615 - val_q7_loss: 0.0109 - val_q8_loss: 0.0031
Epoch 236/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4518 - q0_loss: 0.0025 - q1_loss: 0.0120 - q2_loss: 0.0654 - q3_loss: 0.0882 - q4_loss: 0.1146 - q5_loss: 0.0887 - q6_loss: 0.0655 - q7_loss: 0.0117 - q8_loss: 0.0033 - val_loss: 0.4249 - val_q0_loss: 0.0024 - val_q1_loss: 0.0113 - val_q2_loss: 0.0616 - val_q3_loss: 0.0830 - val_q4_loss: 0.1076 - val_q5_loss: 0.0834 - val_q6_loss: 0.0614 - val_q7_loss: 0.0108 - val_q8_loss: 0.0032
Epoch 237/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4515 - 

Epoch 253/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4540 - q0_loss: 0.0025 - q1_loss: 0.0118 - q2_loss: 0.0657 - q3_loss: 0.0886 - q4_loss: 0.1154 - q5_loss: 0.0889 - q6_loss: 0.0659 - q7_loss: 0.0119 - q8_loss: 0.0033 - val_loss: 0.4255 - val_q0_loss: 0.0024 - val_q1_loss: 0.0112 - val_q2_loss: 0.0619 - val_q3_loss: 0.0831 - val_q4_loss: 0.1081 - val_q5_loss: 0.0830 - val_q6_loss: 0.0617 - val_q7_loss: 0.0108 - val_q8_loss: 0.0033
Epoch 254/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4528 - q0_loss: 0.0025 - q1_loss: 0.0118 - q2_loss: 0.0657 - q3_loss: 0.0882 - q4_loss: 0.1146 - q5_loss: 0.0890 - q6_loss: 0.0657 - q7_loss: 0.0120 - q8_loss: 0.0033 - val_loss: 0.4275 - val_q0_loss: 0.0023 - val_q1_loss: 0.0113 - val_q2_loss: 0.0616 - val_q3_loss: 0.0838 - val_q4_loss: 0.1087 - val_q5_loss: 0.0832 - val_q6_loss: 0.0626 - val_q7_loss: 0.0110 - val_q8_loss: 0.0030
Epoch 255/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4539 - 

Epoch 271/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4516 - q0_loss: 0.0025 - q1_loss: 0.0117 - q2_loss: 0.0654 - q3_loss: 0.0881 - q4_loss: 0.1146 - q5_loss: 0.0886 - q6_loss: 0.0656 - q7_loss: 0.0118 - q8_loss: 0.0032 - val_loss: 0.4242 - val_q0_loss: 0.0023 - val_q1_loss: 0.0111 - val_q2_loss: 0.0616 - val_q3_loss: 0.0831 - val_q4_loss: 0.1076 - val_q5_loss: 0.0831 - val_q6_loss: 0.0614 - val_q7_loss: 0.0108 - val_q8_loss: 0.0031
Epoch 272/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4509 - q0_loss: 0.0024 - q1_loss: 0.0117 - q2_loss: 0.0655 - q3_loss: 0.0880 - q4_loss: 0.1143 - q5_loss: 0.0886 - q6_loss: 0.0656 - q7_loss: 0.0117 - q8_loss: 0.0032 - val_loss: 0.4250 - val_q0_loss: 0.0024 - val_q1_loss: 0.0111 - val_q2_loss: 0.0619 - val_q3_loss: 0.0830 - val_q4_loss: 0.1073 - val_q5_loss: 0.0833 - val_q6_loss: 0.0616 - val_q7_loss: 0.0113 - val_q8_loss: 0.0032
Epoch 273/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4507 - 

Epoch 289/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4508 - q0_loss: 0.0024 - q1_loss: 0.0116 - q2_loss: 0.0653 - q3_loss: 0.0881 - q4_loss: 0.1145 - q5_loss: 0.0885 - q6_loss: 0.0657 - q7_loss: 0.0116 - q8_loss: 0.0032 - val_loss: 0.4245 - val_q0_loss: 0.0022 - val_q1_loss: 0.0109 - val_q2_loss: 0.0618 - val_q3_loss: 0.0830 - val_q4_loss: 0.1074 - val_q5_loss: 0.0833 - val_q6_loss: 0.0616 - val_q7_loss: 0.0111 - val_q8_loss: 0.0031
Epoch 290/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4508 - q0_loss: 0.0025 - q1_loss: 0.0116 - q2_loss: 0.0653 - q3_loss: 0.0881 - q4_loss: 0.1144 - q5_loss: 0.0884 - q6_loss: 0.0655 - q7_loss: 0.0117 - q8_loss: 0.0032 - val_loss: 0.4254 - val_q0_loss: 0.0023 - val_q1_loss: 0.0110 - val_q2_loss: 0.0617 - val_q3_loss: 0.0832 - val_q4_loss: 0.1077 - val_q5_loss: 0.0832 - val_q6_loss: 0.0621 - val_q7_loss: 0.0111 - val_q8_loss: 0.0032
Epoch 291/300
9/9 [==============================] - 0s 4ms/step - loss: 0.4515 - 

In [21]:
# predict validation set
y_pred = model.predict(X_val)
# revert multi output format to (n_samples, quantiles)
y_pred = np.array(list(zip(*y_pred))).squeeze()
y_pred

array([[0.33719954, 0.39197478, 0.4678296 , 0.61838675, 0.7678149 ,
        0.872596  , 1.0035096 , 1.0706043 , 1.3725728 ],
       [0.7454355 , 0.84808606, 0.97579867, 1.2559845 , 1.5239269 ,
        1.7216867 , 1.9954329 , 2.1483293 , 2.235943  ],
       [0.7557534 , 0.85004973, 0.9850245 , 1.2514273 , 1.5165272 ,
        1.7330449 , 1.996386  , 2.1610212 , 2.227574  ],
       [0.33939388, 0.39143473, 0.46943003, 0.6153477 , 0.76358676,
        0.8740001 , 1.0012975 , 1.071847  , 1.3677701 ],
       [0.75136435, 0.85112953, 0.9818233 , 1.2575052 , 1.524983  ,
        1.730236  , 2.0008097 , 2.1585352 , 2.2371795 ],
       [0.74324024, 0.8486259 , 0.9741974 , 1.259024  , 1.5281557 ,
        1.7202816 , 1.9976449 , 2.1470854 , 2.2407465 ],
       [0.3430592 , 0.39443526, 0.4735773 , 0.6187063 , 0.76734126,
        0.8804674 , 1.0074432 , 1.0798583 , 1.3720664 ],
       [0.3519784 , 0.39873692, 0.48253608, 0.62039554, 0.76816285,
        0.8930365 , 1.0148337 , 1.0947906 , 1.3730524 ],


## Test pinball loss

In [22]:
# from https://github.com/keras-team/keras/pull/8033/files
def test_pinball_loss():
    y_pred = K.variable(np.array([0.3, 0.6, 0.1]))
    y_true = K.variable(np.array([0.3, 0.4, 0.5]))
    quantile = 0.25
    loss_fcn = create_pinball_loss(tau=quantile)#losses.PinballLoss(quantile)
    expected_loss = (quantile * 0.4 + (1 - quantile) * 0.2) / 3
    loss = K.eval(loss_fcn(y_true, y_pred))
    assert np.isclose(expected_loss, loss)

test_pinball_loss()